## 2 - Long Short-Term Memory (LSTM) network

This following figure shows the operations of an LSTM-cell.

Similar to the RNN example above, you will start by implementing the LSTM cell for a single time-step. Then you can iteratively call it from inside a for-loop to have it process an input with $T_x$ time-steps. 

### About the gates

#### - Forget gate

For the sake of this illustration, lets assume we are reading words in a piece of text, and want use an LSTM to keep track of grammatical structures, such as whether the subject is singular or plural. If the subject changes from a singular word to a plural word, we need to find a way to get rid of our previously stored memory value of the singular/plural state. In an LSTM, the forget gate lets us do this: 

$$f_{t} = \sigma(W_f[h_{t-1}, x_{t}] + b_f)\tag{1} $$

Here, $W_f$ are weights that govern the forget gate's behavior. We concatenate $[h_{t-1}, x_{t}]$ and multiply by $W_f$. The equation above results in a vector $f_{t}$ with values between 0 and 1. This forget gate vector will be multiplied element-wise by the previous cell state $c_{t-1}$. So if one of the values of $f_{t}$ is 0 (or close to 0) then it means that the LSTM should remove that piece of information (e.g. the singular subject) in the corresponding component of $c_{t-1}$. If one of the values is 1, then it will keep the information. 

#### - Input gate

Once we forget that the subject being discussed is singular, we need to find a way to update it to reflect that the new subject is now plural. Here is the formulat for the input gate: 

$$i_{t} = \sigma(W_i[h_{t-1}, x_{t}] + b_i)\tag{2} $$ 

Similar to the forget gate, here $i_{t}$ is again a vector of values between 0 and 1. This will be multiplied element-wise with $g_{t}$, in order to compute $c_{t}$.

#### - Updating the cell 

To update the new subject we need to create a new vector of numbers that we can add to our previous cell state. The equation we use is: 

$$g_{t} = \tanh(W_c[h_{t-1}, x_{t}] + b_c)\tag{3} $$

Finally, the new cell state is: 

$$ c_{t} = f_{t}* c_{t-1} + i_{t} * g_{t} \tag{4} $$


#### - Output gate

To decide which outputs we will use, we will use the following two formulas: 

$$ o_{t}=  \sigma(W_o[h_{t-1}, x_{t}] + b_o)\tag{5}$$ 
$$ h_{t} = o_{t}* \tanh(c_{t})\tag{6} $$

Where in equation 5 you decide what to output using a sigmoid function and in equation 6 you multiply that by the $\tanh$ of the previous state. 

#### - Output prediction

To define the final prediciton of the LSTM cell, the hidden state will be processed once more. In this case we will use the softmax function:

$$ y_{t} = softmax(W_y h_{t} + b_y)\tag{7} $$


Let's first import all the packages that you will need during this assignment.

In [1]:
import numpy as np
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis = 0)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In this part, you will build a more complex LSTM model, which is better at addressing vanishing gradients. The LSTM will be better able to remember a piece of information and keep it saved for many timesteps. 

### 2.1 - LSTM cell

**Exercise**: Implement the LSTM cell described in the Figure (3).

**Instructions**:
1. Concatenate $h_{t-1}$ and $x_{t}$ in a single matrix: $concat = \begin{bmatrix} h_{t-1} \\ x_{t} \end{bmatrix}$
2. Compute all the formulas 1-6. You can use `sigmoid()` (provided) and `np.tanh()`.
3. Compute the prediction $y_{t}$. You can use `softmax()` (provided).

In [3]:
# GRADED FUNCTION: lstm_cell_forward

def lstm_cell_forward(xt, h_prev, c_prev, parameters):
    """
    Implement a single forward step of the LSTM-cell as described in Figure (4)

    Arguments:
    xt -- your input data at timestep "t", numpy array of shape (n_x, m).
    h_prev -- Hidden state at timestep "t-1", numpy array of shape (n_h, m)
    c_prev -- Memory state at timestep "t-1", numpy array of shape (n_h, m)
    parameters -- python dictionary containing:
                        Wf -- Weight matrix of the forget gate, numpy array of shape (n_h, n_h + n_x)
                        bf -- Bias of the forget gate, numpy array of shape (n_h, 1)
                        Wi -- Weight matrix of the update gate, numpy array of shape (n_h, n_h + n_x)
                        bi -- Bias of the update gate, numpy array of shape (n_h, 1)
                        Wc -- Weight matrix of the first "tanh", numpy array of shape (n_h, n_h + n_x)
                        bc --  Bias of the first "tanh", numpy array of shape (n_h, 1)
                        Wo -- Weight matrix of the output gate, numpy array of shape (n_h, n_h + n_x)
                        bo --  Bias of the output gate, numpy array of shape (n_h, 1)
                        Wy -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_h)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
                        
    Returns:
    h_next -- next hidden state, of shape (n_h, m)
    c_next -- next memory state, of shape (n_h, m)
    yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
    cache -- tuple of values needed for the backward pass, contains (h_next, c_next, h_prev, c_prev, xt, parameters)
    
    Note: ft/it/ot stand for the forget/update/output gates, cct stands for the candidate value (c tilde),
          c stands for the memory value
    """

    # Retrieve parameters from "parameters"
    Wf = parameters["Wf"]
    bf = parameters["bf"]
    Wi = parameters["Wi"]
    bi = parameters["bi"]
    Wc = parameters["Wc"]
    bc = parameters["bc"]
    Wo = parameters["Wo"]
    bo = parameters["bo"]
    Wy = parameters["Wy"]
    by = parameters["by"]
    
    # Retrieve dimensions from shapes of xt and Wy
    n_x, m = xt.shape
    n_y, n_h = Wy.shape

    ### START CODE HERE ###
    # Concatenate a_prev and xt (≈3 lines)
    concat = np.concatenate([h_prev, xt], axis=0)
#     concat = np.zeros((n_x + n_h, m))
#     concat[: n_h, :] = h_prev
#     concat[n_h :, :] = xt

    # Compute values for ft, it, cct, c_next, ot, h_next using the formulas given figure (4) (≈6 lines)
    ft = sigmoid(np.dot(Wf, concat) + bf)
    it = sigmoid(np.dot(Wi, concat) + bi)
    cct = np.tanh(np.dot(Wc, concat) + bc)
    c_next = ft * c_prev + it * cct
    ot = sigmoid(np.dot(Wo, concat) + bo)
    h_next = ot * np.tanh(c_next)
    
    # Compute prediction of the LSTM cell (≈1 line)
    yt_pred = softmax(np.dot(Wy, h_next) + by)
    ### END CODE HERE ###

    # store values needed for backward propagation in cache
    cache = (h_next, c_next, h_prev, c_prev, ft, it, cct, ot, xt, parameters)

    return h_next, c_next, yt_pred, cache

In [5]:
np.random.seed(1)
xt = np.random.randn(3,10)
h_prev = np.random.randn(5,10)
c_prev = np.random.randn(5,10)
Wf = np.random.randn(5, 5+3)
bf = np.random.randn(5,1)
Wi = np.random.randn(5, 5+3)
bi = np.random.randn(5,1)
Wo = np.random.randn(5, 5+3)
bo = np.random.randn(5,1)
Wc = np.random.randn(5, 5+3)
bc = np.random.randn(5,1)
Wy = np.random.randn(2,5)
by = np.random.randn(2,1)

parameters = {"Wf": Wf, "Wi": Wi, "Wo": Wo, "Wc": Wc, "Wy": Wy, "bf": bf, "bi": bi, "bo": bo, "bc": bc, "by": by}

h_next, c_next, yt, cache = lstm_cell_forward(xt, h_prev, c_prev, parameters)
print("h_next[4] = ", h_next[4])
print("h_next.shape = ", c_next.shape)
print("c_next[2] = ", c_next[2])
print("c_next.shape = ", c_next.shape)
print("yt[1] =", yt[1])
print("yt.shape = ", yt.shape)
print("cache[1][3] =", cache[1][3])
print("len(cache) = ", len(cache))

h_next[4] =  [-0.66408471  0.0036921   0.02088357  0.22834167 -0.85575339  0.00138482
  0.76566531  0.34631421 -0.00215674  0.43827275]
h_next.shape =  (5, 10)
c_next[2] =  [ 0.63267805  1.00570849  0.35504474  0.20690913 -1.64566718  0.11832942
  0.76449811 -0.0981561  -0.74348425 -0.26810932]
c_next.shape =  (5, 10)
yt[1] = [0.79913913 0.15986619 0.22412122 0.15606108 0.97057211 0.31146381
 0.00943007 0.12666353 0.39380172 0.07828381]
yt.shape =  (2, 10)
cache[1][3] = [-0.16263996  1.03729328  0.72938082 -0.54101719  0.02752074 -0.30821874
  0.07651101 -1.03752894  1.41219977 -0.37647422]
len(cache) =  10


**Expected Output**:

<table>
    <tr>
        <td>
            **h_next[4]**:
        </td>
        <td>
           [-0.66408471  0.0036921   0.02088357  0.22834167 -0.85575339  0.00138482
  0.76566531  0.34631421 -0.00215674  0.43827275]
        </td>
    </tr>
        <tr>
        <td>
            **h_next.shape**:
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **c_next[2]**:
        </td>
        <td>
           [ 0.63267805  1.00570849  0.35504474  0.20690913 -1.64566718  0.11832942
  0.76449811 -0.0981561  -0.74348425 -0.26810932]
        </td>
    </tr>
        <tr>
        <td>
            **c_next.shape**:
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **yt[1]**:
        </td>
        <td>
           [ 0.79913913  0.15986619  0.22412122  0.15606108  0.97057211  0.31146381
  0.00943007  0.12666353  0.39380172  0.07828381]
        </td>
    </tr>
        <tr>
        <td>
            **yt.shape**:
        </td>
        <td>
           (2, 10)
        </td>
    </tr>
    <tr>
        <td>
            **cache[1][3]**:
        </td>
        <td>
           [-0.16263996  1.03729328  0.72938082 -0.54101719  0.02752074 -0.30821874
  0.07651101 -1.03752894  1.41219977 -0.37647422]
        </td>
    </tr>
        <tr>
        <td>
            **len(cache)**:
        </td>
        <td>
           10
        </td>
    </tr>

</table>

### 2.2 - Forward pass for LSTM

Now that you have implemented one step of an LSTM, you can now iterate this over this using a for-loop to process a sequence of $T_x$ inputs. 

**Exercise:** Implement `lstm_forward()` to run an LSTM over $T_x$ time-steps. 

**Note**: $c_{0}$ is initialized with zeros.

In [4]:
# GRADED FUNCTION: lstm_forward

def lstm_forward(x, h0, parameters):
    """
    Implement the forward propagation of the recurrent neural network using an LSTM-cell described in Figure (3).

    Arguments:
    x -- Input data for every time-step, of shape (n_x, m, T_x).
    h0 -- Initial hidden state, of shape (n_h, m)
    parameters -- python dictionary containing:
                        Wf -- Weight matrix of the forget gate, numpy array of shape (n_h, n_h + n_x)
                        bf -- Bias of the forget gate, numpy array of shape (n_h, 1)
                        Wi -- Weight matrix of the update gate, numpy array of shape (n_h, n_h + n_x)
                        bi -- Bias of the update gate, numpy array of shape (n_h, 1)
                        Wc -- Weight matrix of the first "tanh", numpy array of shape (n_h, n_h + n_x)
                        bc -- Bias of the first "tanh", numpy array of shape (n_h, 1)
                        Wo -- Weight matrix of the output gate, numpy array of shape (n_h, n_h + n_x)
                        bo -- Bias of the output gate, numpy array of shape (n_h, 1)
                        Wy -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_h)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
                        
    Returns:
    h -- Hidden states for every time-step, numpy array of shape (n_h, m, T_x)
    y -- Predictions for every time-step, numpy array of shape (n_y, m, T_x)
    caches -- tuple of values needed for the backward pass, contains (list of all the caches, x)
    """

    # Initialize "caches", which will track the list of all the caches
    caches = []
    
    ### START CODE HERE ###
    # Retrieve dimensions from shapes of x and parameters['Wy'] (≈2 lines)
    n_x, m, T_x = x.shape
    n_y, n_h = parameters['Wy'].shape
    
    # initialize "a", "c" and "y" with zeros (≈3 lines)
    h = np.zeros((n_h, m, T_x))
    c = np.zeros((n_h, m, T_x))
    y = np.zeros((n_y, m, T_x))
    
    # Initialize a_next and c_next (≈2 lines)
    h_next = h0
    c_next = np.zeros((n_h, m))
    
    # loop over all time-steps
    for t in range(T_x):
        # Update next hidden state, next memory state, compute the prediction, get the cache (≈1 line)
        h_next, c_next, yt, cache = lstm_cell_forward(x[:,:,t], h_next, c_next, parameters)
        # Save the value of the new "next" hidden state in a (≈1 line)
        h[:,:,t] = h_next
        # Save the value of the prediction in y (≈1 line)
        y[:,:,t] = yt
        # Save the value of the next cell state (≈1 line)
        c[:,:,t]  = c_next
        # Append the cache into caches (≈1 line)
        caches.append(cache)
        
    ### END CODE HERE ###
    
    # store values needed for backward propagation in cache
    caches = (caches, x)

    return h, y, c, caches

In [5]:
np.random.seed(1)
x = np.random.randn(3,10,7)
h0 = np.random.randn(5,10)
Wf = np.random.randn(5, 5+3)
bf = np.random.randn(5,1)
Wi = np.random.randn(5, 5+3)
bi = np.random.randn(5,1)
Wo = np.random.randn(5, 5+3)
bo = np.random.randn(5,1)
Wc = np.random.randn(5, 5+3)
bc = np.random.randn(5,1)
Wy = np.random.randn(2,5)
by = np.random.randn(2,1)

parameters = {"Wf": Wf, "Wi": Wi, "Wo": Wo, "Wc": Wc, "Wy": Wy, "bf": bf, "bi": bi, "bo": bo, "bc": bc, "by": by}

h, y, c, caches = lstm_forward(x, h0, parameters)
print("h[4][3][6] = ", h[4][3][6])
print("h.shape = ", h.shape)
print("y[1][4][3] =", y[1][4][3])
print("y.shape = ", y.shape)
print("caches[1][1[1]] =", caches[1][1][1])
print("c[1][2][1]", c[1][2][1])
print("len(caches) = ", len(caches))

h[4][3][6] =  0.1721177675329167
h.shape =  (5, 10, 7)
y[1][4][3] = 0.9508734618501101
y.shape =  (2, 10, 7)
caches[1][1[1]] = [ 0.82797464  0.23009474  0.76201118 -0.22232814 -0.20075807  0.18656139
  0.41005165]
c[1][2][1] -0.8555449167181983
len(caches) =  2


**Expected Output**:

<table>
    <tr>
        <td>
            **a[4][3][6]** =
        </td>
        <td>
           0.172117767533
        </td>
    </tr>
        <tr>
        <td>
            **a.shape** =
        </td>
        <td>
           (5, 10, 7)
        </td>
    </tr>
        <tr>
        <td>
            **y[1][4][3]** =
        </td>
        <td>
           0.95087346185
        </td>
    </tr>
        <tr>
        <td>
            **y.shape** =
        </td>
        <td>
           (2, 10, 7)
        </td>
    </tr>
        <tr>
        <td>
            **caches[1][1][1]** =
        </td>
        <td>
           [ 0.82797464  0.23009474  0.76201118 -0.22232814 -0.20075807  0.18656139
              0.41005165 ]
            </td>
    </tr>
               <tr>
        <td>
            **c[1][2][1]** =
        </td>
        <td>
           -0.855544916718
                   </td>
    </tr>
        <tr>
        <td>
            **len(caches)** =
        </td>
        <td>
           2
        </td>
    </tr>


</table>

Congratulations! You have now implemented the forward passes for the basic RNN and the LSTM. When using a deep learning framework, implementing the forward pass is sufficient to build systems that achieve great performance. 


### Congratulations !

Congratulations on completing this assignment. You now understand how recurrent neural networks work! 

